In [1]:
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
market = Market()
sec = SEC()

In [3]:
financial_db = ADatabase("strategy_financial")

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
prices = market.retrieve("alpha_prices")
market.disconnect()

In [5]:
prices = p.column_date_processing(prices)
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["year"] = [x.year for x in prices["date"]]
prices["close"] = [float(x) for x in prices["close"]]

In [42]:
sec.connect()
training_data = []
training_columns = {}
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        ticker_financials = filing.rename(columns={"filed":"date"})
        ticker_financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in ticker_financials["date"]]
        ticker_financials["year"] = [x.year - 1 for x in ticker_financials["date"]]
        ticker_financials["quarter"] = [x.quarter for x in ticker_financials["date"]]
        ticker_financials = p.column_date_processing(ticker_financials)
        for col in ticker_financials.columns:
            if col in training_columns.keys():
                training_columns[col] += 1
            else:
                training_columns[col] = 0
        ticker_financials["ticker"] = ticker
        training_data.append(ticker_financials)
    except Exception as e:
        print(str(e))
sec.disconnect()

 39%|██████████████████████████████████████████████████████████▉                                                                                            | 197/505 [01:07<02:24,  2.13it/s]

'date'


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 417/505 [02:19<00:39,  2.23it/s]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:46<00:00,  3.02it/s]


In [43]:
unrequired = ['adsh',
 'cik',
 'name',
 'sic',
 'countryba',
 'stprba',
 'cityba',
 'zipba',
 'bas1',
 'bas2',
 'baph',
 'countryma',
 'stprma',
 'cityma',
 'zipma',
 'mas1',
 'mas2',
 'countryinc',
 'stprinc',
 'ein',
 'former',
 'changed',
 'afs',
 'wksi',
 'fye',
 'form',
 'period',
 'fy',
 'fp',
 'date',
 'accepted',
 'prevrpt',
 'detail',
 'instance',
 'nciks',
 'aciks']

In [44]:
factors = [x for x in training_columns if x not in unrequired and training_columns[x] >= 425]
factors.extend(["ticker","y"])

In [45]:
factors

['accumulateddepreciationdepletionandamortizationpropertyplantandequipment',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'assets',
 'assetscurrent',
 'cashandcashequivalentsatcarryingvalue',
 'cashandcashequivalentsperiodincreasedecrease',
 'commonstockparorstatedvaluepershare',
 'commonstocksharesissued',
 'commonstockvalue',
 'deferredincometaxexpensebenefit',
 'earningspersharebasic',
 'earningspersharediluted',
 'entitycommonstocksharesoutstanding',
 'entitypublicfloat',
 'goodwill',
 'incometaxexpensebenefit',
 'liabilitiesandstockholdersequity',
 'liabilitiescurrent',
 'netcashprovidedbyusedinfinancingactivities',
 'netcashprovidedbyusedininvestingactivities',
 'netcashprovidedbyusedinoperatingactivities',
 'netincomeloss',
 'paymentsforrepurchaseofcommonstock',
 'paymentstoacquirebusinessesnetofcashacquired',
 'propertyplantandequipmentgross',
 'propertyplantandequipmentnet',
 'retainedearningsaccumulateddeficit',
 'sharebasedcompensation',
 'stockholdersequity',
 'weig

In [46]:
sec.connect()
training_data = []
training_columns = {}
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        ticker_financials = filing.rename(columns={"filed":"date"})
        ticker_financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in ticker_financials["date"]]
        ticker_financials["year"] = [x.year - 1 for x in ticker_financials["date"]]
        ticker_financials["quarter"] = [x.quarter for x in ticker_financials["date"]]
        ticker_financials = p.column_date_processing(ticker_financials)
        ticker_financials["ticker"] = ticker
        for col in factors:
            if col not in ticker_financials.columns:
                ticker_financials[col] = np.NAN
        ticker_financials["y"] = ticker_financials["earningspersharebasic"].shift(-1)
        training_data.extend(ticker_financials[factors].to_dict("records"))
    except Exception as e:
        print(str(e))
sec.disconnect()

 39%|██████████████████████████████████████████████████████████▉                                                                                            | 197/505 [01:08<02:26,  2.11it/s]

'date'


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 417/505 [02:23<00:41,  2.11it/s]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:52<00:00,  2.93it/s]


In [47]:
data = pd.DataFrame(training_data)

In [49]:
data.fillna(0,inplace=True)

In [53]:
start_year = 2016
end_year = 2022
performance = []
sim = []
financial_db.connect()
for year in tqdm(range(start_year,end_year)):
    try:
        training_data = data[(data["year"]>=year-6) & (data["year"]<year)].dropna().reset_index(drop=True)
        if training_data.index.size > 100:
            prediction_data = data[(data["year"]==year)].dropna().reset_index(drop=True)
            final = [x for x in factors if x not in ["year","quarter","ticker","y"]]
            refined_data = {"X":training_data[final],"y":training_data[["y"]]}
            models = m.regression(refined_data)
            models["year"] = year
            for row in models.iterrows():
                api = row[1]["api"]
                prediction = row[1]["model"].predict(prediction_data[final])
                prediction_data[f"{api}_prediction"] = prediction
                prediction_data[f"{api}_score"] = row[1]["score"]
            included_columns = ["year","quarter","ticker"]
            included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
            sim.append(prediction_data[included_columns])
    except Exception as e:
        print(year,str(e))
financial_db.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:48<00:00, 38.08s/it]


In [54]:
simulation = pd.concat(sim).groupby(["year","quarter","ticker"]).mean().reset_index()
simulation["average_prediction"] = [sum([row[1][col] for col in simulation.columns if "prediction" in col]) for row in simulation.iterrows()]

In [55]:
simulation = prices.merge(simulation,on=["year","quarter","ticker"]).dropna()

In [56]:
simulation["pe"] = simulation["close"] / simulation["average_prediction"]

In [57]:
simulation

,date,open,high,low,close,volume,ticker,quarter,year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,average_prediction,pe
0,2016-01-04,148.0500,148.3200,145.4000,146.82,3277188,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.037281
1,2016-01-05,146.8200,147.5000,145.6100,147.46,2688082,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.037444
2,2016-01-06,145.5900,145.7600,143.4200,144.49,2997126,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.036690
3,2016-01-07,142.5200,143.1300,140.6300,140.97,3553519,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.035796
4,2016-01-08,141.3600,142.5000,140.2200,140.49,2663879,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.035674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635169,2021-03-25,158.3200,158.3200,154.8800,157.88,1081864,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,42.449120
635170,2021-03-26,158.3500,161.4900,157.2900,161.32,655775,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,43.374031
635171,2021-03-29,160.1900,161.2700,158.9150,160.21,667083,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,43.075586
635172,2021-03-30,159.0300,161.5100,158.8100,161.22,762880,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,43.347144


In [58]:
industry_stuff = simulation.merge(sp5.rename(columns={"Symbol":"ticker"}),on="ticker").groupby(["year","quarter","GICS Sector"]).mean().reset_index()[["year","quarter","GICS Sector","pe"]]

In [59]:
sim_final = simulation.merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","GICS Sector"]],on="ticker",how="left").merge(industry_stuff.rename(columns={"pe":"industry_pe"}),on=["year","quarter","GICS Sector"],how="left")

In [60]:
sim_final["delta"] = (sim_final["pe"] - sim_final["industry_pe"]) / sim_final["industry_pe"]

In [63]:
sim_final["delta"].mean()

2.0314836692628192e-17

In [66]:
sim_final

,date,open,high,low,close,volume,ticker,quarter,year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,average_prediction,pe,GICS Sector,industry_pe,delta
0,2016-01-04,148.0500,148.3200,145.4000,146.82,3277188,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.037281,Industrials,-0.047684,-0.218164
1,2016-01-05,146.8200,147.5000,145.6100,147.46,2688082,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.037444,Industrials,-0.047684,-0.214756
2,2016-01-06,145.5900,145.7600,143.4200,144.49,2997126,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.036690,Industrials,-0.047684,-0.230572
3,2016-01-07,142.5200,143.1300,140.6300,140.97,3553519,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.035796,Industrials,-0.047684,-0.249316
4,2016-01-08,141.3600,142.5000,140.2200,140.49,2663879,MMM,1,2016,-1692.014783,-0.004197,-2328.799561,-149.450267,82.654953,-6.375688,-3938.159391,-0.035674,Industrials,-0.047684,-0.251872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635169,2021-03-25,158.3200,158.3200,154.8800,157.88,1081864,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,42.449120,Health Care,37.273751,0.138848
635170,2021-03-26,158.3500,161.4900,157.2900,161.32,655775,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,43.374031,Health Care,37.273751,0.163662
635171,2021-03-29,160.1900,161.2700,158.9150,160.21,667083,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,43.075586,Health Care,37.273751,0.155655
635172,2021-03-30,159.0300,161.5100,158.8100,161.22,762880,ZBH,1,2021,1.987390,-0.000323,0.818582,-349870.400822,0.913304,-744629.270232,3.719276,43.347144,Health Care,37.273751,0.162940


In [ ]:
positions = 5
start_date = sim_final["date"].min()
end_date = sim_final["date"].max()
reqs = [15,30,50,90]
limit = 90
values = [True,False]
trades = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            date = start_date
            iterration_sim = sim_final.copy()
            if value:
                iterration_sim["delta"] = iterration_sim["delta"] * -1
            while date < end_date:
                try:
                    todays_recs = iterration_sim[iterration_sim["date"]==date]
                    todays_recs.sort_values("delta",ascending=False,inplace=True)
                    if todays_recs.index.size > 0:
                        offering = todays_recs.iloc[position]
                        if offering["delta"] > req:
                            trade = offering
                            ticker = trade["ticker"]
                            buy_price = trade["close"]
                            exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:limit]
                            exits["gains"] = (exits["close"] - buy_price) / buy_price
                            exit = exits.sort_values("gains",ascending=False).iloc[0]
                            trade["sell_price"] = exit["close"]
                            trade["sell_date"] = exit["date"]
                            date = exit["date"] + timedelta(days=1)
                            trade["value"] = value
                            trade["req"] = req
                            trade["position"] = position
                            trades.append(trade)
                        else:
                            date = date + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)
                except Exception as e:
                    print(str(e))
                    date = date+timedelta(days=1)

 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                      | 3/4 [00:36<00:12, 12.55s/it]

In [ ]:
t = pd.DataFrame(trades)

In [ ]:
t["delta"] =  (t["sell_price"] - t["close"]) / t["close"]

In [ ]:
analysis = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            position_trades = t[(t["value"]==value) & (t["req"]==req) & (t["position"]==position)].sort_values("date")
            position_trades["delta"] = (position_trades["sell_price"] - position_trades["close"]) / position_trades["close"]
            initial = 100 / positions
            for delta in position_trades["delta"]:
                initial = initial * (1+delta)
            analysis.append({
                "value":value,
                "req":req,
                "position":position,
                "pv":initial
            })
            

In [ ]:
complete = pd.DataFrame(analysis).pivot_table(index=["value","req"],columns="position",values="pv").reset_index()

In [ ]:
complete["pv"] = [sum([row[1][i] for i in range(5)]) for row in complete.iterrows()]

In [ ]:
complete.sort_values("pv",ascending=False)